# Import

In [1]:
import sys
%matplotlib inline
SCRIPTS_PATH = '../scripts'
if SCRIPTS_PATH not in sys.path:
    sys.path.append(SCRIPTS_PATH)

import numpy as np
import pandas as pd
import matplotlib.pylab as plt
import regression_model
import NNModel
from NNModel import *
from window_aggregation import *

Using TensorFlow backend.


# Load test data

In [48]:
test_df = pd.read_csv('../data/test.csv')

In [38]:
full_df = pd.read_csv('../data/dataset1_v2.csv')

In [43]:
prods = full_df['PRODUCT'].unique().tolist()
selected_prods = np.random.choice(prods, 30)
out_df = full_df[full_df['PRODUCT'].isin(selected_prods)]
out_df.to_csv('../data/test.csv', index=False)

# Init models

In [3]:
nn_model = nn_init('../model/')

/Library/Frameworks/Python.framework/Versions/3.5/lib/python3.5/site-packages/sklearn/base.py:312: UserWarning: Trying to unpickle estimator OneHotEncoder from version 0.18.1 when using version 0.19.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


# Predict

In [50]:
products = test_df['PRODUCT'].unique()
for product in products:
    comments = test_df[test_df['PRODUCT'] == product]['TEXT'].tolist()
    prod_info = test_df[test_df['PRODUCT'] == product].head(1)
    prod_name = prod_info['NAME'].tolist()[0].strip() if len(prod_info['NAME']) > 0 else ''
    print('Продукт %d, %s' % (product, prod_name))
    wnd_agg = aggregate(comments, num_comments=3, model=nn_model, wnd_l=12, stride=6, positive_q=(0.8, 0.95), negative_threshold=3)
    print('\tПокупатели отмечают такие положительные моменты: ')
    if len(wnd_agg['+']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['+']:
            print('\t\t...%s...' % com)
            
    print('\tПокупатели отмечают такие отрицательные моменты: ')
    if len(wnd_agg['-']) == 0:
        print('\t\t---')
    else:
        for com in wnd_agg['-']:
            print('\t\t...%s...' % com)
    print(' ')

Продукт 10000838, Автомобильная магнитола с CD MP3 Pioneer DEH-9300SD
	Покупатели замечают такие положительные моменты: 
		...выглядит очень стильно и добротно ...
		...читает, что не может не радовать) главное прикупить пульт и про якобы ...
		...аппарат отличный ...
	Покупатели замечают такие отрицательные моменты: 
		---
 
Продукт 10001253, Телевизор LG 32 LW4500
	Покупатели замечают такие положительные моменты: 
		...изображение отличное ...
		...настройки простые и удобные настроил тв не открывая инструкцию ...
		...звука), все работает, легко и быстро переключается с аппарата на аппарат ...
	Покупатели замечают такие отрицательные моменты: 
		---
 
Продукт 10005021, Автомобильный усилитель (1 канал) Pioneer GM-D8601
	Покупатели замечают такие положительные моменты: 
		...и самое главное мощный ...
		...выдает четкие басы, проигрывает все виды музыкальных произведений ...
		...очень простая навигация, с легкостью выставил настройки под себя ...
	Покупатели замечают такие отрицател

	Покупатели замечают такие положительные моменты: 
		...на глаза мне попал thrustmaster gpx lightback ferrari , яркая упаковка ...
		...присутствует , форма прежняя, как и на классическом удобно вливается в руки ...
	Покупатели замечают такие отрицательные моменты: 
		---
 
Продукт 40064074, Видеоигра для PS4 . Pro Evolution Soccer 2016
	Покупатели замечают такие положительные моменты: 
		...игра лучшая в своём жанре ...
	Покупатели замечают такие отрицательные моменты: 
		---
 
Продукт 50034449, Мультимедийный проигрыватель ASUS HDP-R3
	Покупатели замечают такие положительные моменты: 
		...сразу обновил прошивку ...
		...корень флешки, вставляем в медиапроигрыватель, входим в меню обновление – где он ...
		...жмём ок! 2 минуты – и готово ...
	Покупатели замечают такие отрицательные моменты: 
		...сети проблем тоже нет,по вай фай скорость так и не поднялась больше ...
		...скорость так и не поднялась больше 7-9 мбит(даже при максимальном приближении роутера ...
		...тормозит и так же 